In [1]:
from IPython.display import display
import mangoes
import nltk
import string 
import pprint 
import os 
import datetime 

Read more for 

https://github.com/UniversalDependencies/UD_English-EWT


In [2]:
date = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")

In [3]:
# Or specify date to saved ones 

In [4]:
OUTPUT_PATH = os.path.join(os.path.abspath(''), "output/{}".format(date))
# if not os.path.exists(OUTPUT_PATH):
#     print("made a dir: ", OUTPUT_PATH)
#     os.makedirs(OUTPUT_PATH)   

In [5]:
date

'2021-03-16-23-12'

# Build Embedding 
### Corpus

In [6]:
UD_English_EWT = "../../UD_English-EWT-master/corpus"
# UD_English_EWT =  "../../UD_English-EWT-master/corpus/en_ewt-ud-train.conllu"
# WIKI = "../../wikipedia_data/treebank.conllu"

In [7]:
corpus = mangoes.Corpus(UD_English_EWT, 
                        reader=mangoes.corpus.CONLLU, 
                        language="English",
                        lower=True, 
                        ignore_punctuation=False)

Counting words: 0it [00:00, ?it/s]

In [8]:
print("{} sentences, {} words".format(corpus.nb_sentences, corpus.size))

29887 sentences, 615697 words


In [9]:
corpus.describe()

Name: ../../UD_English-EWT-master/corpus
Language: English
Reader: <class 'mangoes.utils.reader.ConllUSentenceGenerator'>
Parameters:
	- lower : True
	- digit : False
	- ignore_punctuation : False
Size:
	- sentences: 29887
	- total number of tokens: 615697
	- number of unique tokens: 48789


In [10]:
# Save corpus 
# corpus_metadata = os.path.join(OUTPUT_PATH, ".corpus")
# print("Saved Corpus")
# corpus.save_metadata(corpus_metadata)

In [11]:
# or load 
# corpus = mangoes.Corpus.load_from_metadata(corpus_metadata)

### Vocabulary
#### -- Target Words --

In [12]:
stopwords_filter_lemma = mangoes.corpus.remove_elements(nltk.corpus.stopwords.words('english'), attribute="lemma")

#### Option 1: Use Lemma and POS as targetwords

In [13]:
# target_vocabulary = corpus.create_vocabulary(attributes=("lemma","POS"), 
#                                               filters = [ stopwords_filter_lemma, 
#                                                          mangoes.corpus.remove_most_frequent(100),
#                                                          mangoes.corpus.remove_least_frequent(2)])

#### Option 2:

In [14]:
# option for only keeping lemma 
stopwords_filter = mangoes.corpus.remove_elements(nltk.corpus.stopwords.words('english'))

In [15]:
# target_vocabulary = corpus.create_vocabulary(attributes="lemma", 
#                                               filters = [ stopwords_filter, 
#                                                          mangoes.corpus.remove_most_frequent(100),
#                                                          mangoes.corpus.remove_least_frequent(2)])

#### Option 3: Use Lemma and POS as targetwords + keep only POS of "NOUN", "VERB", "ADJ"

In [16]:
pos_filter_target = mangoes.corpus.filter_by_attribute("POS", ["NOUN", "VERB", "ADJ"])

In [17]:
# must include POS as one of attributes if POS filter is used
target_vocabulary = corpus.create_vocabulary(attributes=("lemma","POS"), 
                                            filters = [ stopwords_filter_lemma, 
                                                        pos_filter_target,
                                                        mangoes.corpus.remove_most_frequent(100),
                                                        mangoes.corpus.remove_least_frequent(2)],)

#### Check target vocaulary

In [18]:
print(f"{len(target_vocabulary)} words will be used as target vocabulary")

11077 words will be used as target vocabulary


In [19]:
# check all the target vocabulary
target_vocabulary._index_word

[Token(lemma='story', POS='NOUN'),
 Token(lemma='attack', POS='NOUN'),
 Token(lemma='social', POS='ADJ'),
 Token(lemma='bad', POS='ADJ'),
 Token(lemma='market', POS='NOUN'),
 Token(lemma='house', POS='NOUN'),
 Token(lemma='territory', POS='NOUN'),
 Token(lemma='original', POS='ADJ'),
 Token(lemma='claim', POS='VERB'),
 Token(lemma='style', POS='NOUN'),
 Token(lemma='horse', POS='NOUN'),
 Token(lemma='text', POS='NOUN'),
 Token(lemma='western', POS='ADJ'),
 Token(lemma='decision', POS='NOUN'),
 Token(lemma='true', POS='ADJ'),
 Token(lemma='friendly', POS='ADJ'),
 Token(lemma='file', POS='NOUN'),
 Token(lemma='choose', POS='VERB'),
 Token(lemma='discuss', POS='VERB'),
 Token(lemma='mention', POS='VERB'),
 Token(lemma='propose', POS='VERB'),
 Token(lemma='excellent', POS='ADJ'),
 Token(lemma='surface', POS='NOUN'),
 Token(lemma='popular', POS='ADJ'),
 Token(lemma='expect', POS='VERB'),
 Token(lemma='happen', POS='VERB'),
 Token(lemma='german', POS='ADJ'),
 Token(lemma='object', POS='NOUN'

#### -- Context Words --

#### Option 1: Only keep lemma 

In [20]:
context_vocabulary = corpus.create_vocabulary(attributes=("lemma"), 
                                              filters = [ stopwords_filter])

#### Option 2: Keep lemma and POS with "VERB" or "ADJ"

In [21]:
pos_filter_context = mangoes.corpus.filter_by_attribute("POS",["NOUN", "VERB", "ADJ"])

In [22]:
context_vocabulary = corpus.create_vocabulary(attributes=("lemma","POS"), 
                                            filters = [ stopwords_filter_lemma,
                                                        pos_filter_context])

#### Check context vocabulary

In [23]:
print(f"{len(context_vocabulary)} words will be used as target vocabulary")

21237 words will be used as target vocabulary


In [24]:
context_vocabulary._index_word

[Token(lemma='use', POS='VERB'),
 Token(lemma='time', POS='NOUN'),
 Token(lemma='make', POS='VERB'),
 Token(lemma='year', POS='NOUN'),
 Token(lemma='go', POS='VERB'),
 Token(lemma='know', POS='VERB'),
 Token(lemma='get', POS='VERB'),
 Token(lemma='take', POS='VERB'),
 Token(lemma='include', POS='VERB'),
 Token(lemma='say', POS='VERB'),
 Token(lemma='many', POS='ADJ'),
 Token(lemma='first', POS='ADJ'),
 Token(lemma='give', POS='VERB'),
 Token(lemma='number', POS='NOUN'),
 Token(lemma='see', POS='VERB'),
 Token(lemma='call', POS='VERB'),
 Token(lemma='day', POS='NOUN'),
 Token(lemma='find', POS='VERB'),
 Token(lemma='state', POS='NOUN'),
 Token(lemma='people', POS='NOUN'),
 Token(lemma='new', POS='ADJ'),
 Token(lemma='become', POS='VERB'),
 Token(lemma='good', POS='ADJ'),
 Token(lemma='come', POS='VERB'),
 Token(lemma='want', POS='VERB'),
 Token(lemma='place', POS='NOUN'),
 Token(lemma='great', POS='ADJ'),
 Token(lemma='service', POS='NOUN'),
 Token(lemma='work', POS='NOUN'),
 Token(lemm

### Context 

#### Option 1: consider dependency relation  as undirected by setting directed parameter 

In [25]:
# dependency_context = mangoes.context.DependencyBasedContext(entity=("lemma","POS"), 
#                                                             labels=True,
#                                                             collapse=True, 
#                                                             vocabulary=context_vocabulary,
#                                                             directed=False)

#### Option 2: consider only dependency relation of "advmod" or "nsubj"
Note if collapse is set to True, then case will be included as well. 


This will result in changed number of context words when co-occurrence matrix is built 

In [26]:
# dependency_context = mangoes.context.DependencyBasedContext(entity=("lemma","POS"), 
#                                                             labels=True,
#                                                             collapse=True, 
#                                                             vocabulary=context_vocabulary,
#                                                             deprel_keep = ("advmod", "nsubj"),
#                                                             directed=False)

#### Option 3 adding depth to consider indirected dependency relation

In [27]:
# dependency_context = mangoes.context.DependencyBasedContext(entity=("lemma","POS"), 
#                                                             labels=True,
#                                                             collapse=True, 
#                                                             vocabulary=context_vocabulary,
#                                                             directed=False,
#                                                             depth=2
#                                                             )

#### Option 4: Combining option 2 and 3

If `depth` > 1 and `deprel_keep` is specified then `deprel_keep` will be counted as long as indirected dependency relations has one relation from `deprel_keep`.


eg. If `depth=2` and `deprel_keep=("nsubj")` then  `{'lemma': 'muslim', 'POS': 'ADJ'}/nsubj_amod",` will be included as context words since one of the dependency relation `nsubj` is in `deprel_keep`

In [28]:
dependency_context = mangoes.context.DependencyBasedContext(entity=("lemma","POS"), 
                                                            labels=True,
                                                            collapse=True, 
                                                            vocabulary=context_vocabulary,
                                                            directed=False,
                                                            deprel_keep=("advmod", "nsubj"),
                                                            depth=2
                                                            )

In [29]:
# # save vocabularies 
# target_vocabulary_file_name = "vocabulary_{}_target_words".format(len(target_vocabulary))
# context_vocabulary_file_name = "vocabulary_{}_context_words".format(len(context_vocabulary))

# or load by specify saved vocabulary files 

In [30]:
# # save
# target_vocabulary.save(OUTPUT_PATH, name=target_vocabulary_file_name)
# context_vocabulary.save(OUTPUT_PATH, name=context_vocabulary_file_name)

In [31]:
# load vocabs
# target_vocabulary = mangoes.Vocabulary.load("output", target_vocabulary_file_name)
# context_vocabulary = mangoes.Vocabulary.load("output", context_vocabulary_file_name)

### Cooccurrence Matrix 

In [32]:
coocc_count = mangoes.counting.count_cooccurrence(corpus,  
                                                target_vocabulary, 
                                                context=dependency_context,
                                                )

coocc_count.pprint(display=display)

,,"{'lemma': 'detonate', 'POS': 'VERB'}/nsubj","{'lemma': 'guerrilla', 'POS': 'NOUN'}/nsubj","{'lemma': 'hope', 'POS': 'VERB'}/advcl_nsubj","{'lemma': 'tradeoff', 'POS': 'NOUN'}/nsubj","{'lemma': 'move', 'POS': 'NOUN'}/nsubj","{'lemma': 'suckup', 'POS': 'NOUN'}/acl_nsubj","{'lemma': 'people', 'POS': 'NOUN'}/conj_nsubj","{'lemma': 'suckup', 'POS': 'NOUN'}/nsubj","{'lemma': 'people', 'POS': 'NOUN'}/nsubj","{'lemma': 'remain', 'POS': 'VERB'}/nsubj_amod",...,"{'lemma': 'attack', 'POS': 'NOUN'}/conj_nsubj","{'lemma': 'derive', 'POS': 'VERB'}/nsubj_case_of","{'lemma': 'story', 'POS': 'NOUN'}/nsubj_case_of","{'lemma': 'usage', 'POS': 'NOUN'}/conj_nsubj","{'lemma': 'boethius', 'POS': 'NOUN'}/nsubj","{'lemma': 'translate', 'POS': 'VERB'}/conj_nsubj","{'lemma': 'fact', 'POS': 'NOUN'}/nsubj_case_of","{'lemma': 'formalist', 'POS': 'ADJ'}/nsubj_amod","{'lemma': 'truth', 'POS': 'NOUN'}/acl:relcl_nsubj","{'lemma': 'statement', 'POS': 'NOUN'}/acl:relcl_nsubj"
story,NOUN,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
attack,NOUN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
social,ADJ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bad,ADJ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
market,NOUN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
punch,VERB,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
nursery,NOUN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
rhyme,NOUN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
axiomatize,VERB,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
print("target words")
coocc_count.words[:20]

target words


[Token(lemma='story', POS='NOUN'),
 Token(lemma='attack', POS='NOUN'),
 Token(lemma='social', POS='ADJ'),
 Token(lemma='bad', POS='ADJ'),
 Token(lemma='market', POS='NOUN'),
 Token(lemma='house', POS='NOUN'),
 Token(lemma='territory', POS='NOUN'),
 Token(lemma='original', POS='ADJ'),
 Token(lemma='claim', POS='VERB'),
 Token(lemma='style', POS='NOUN'),
 Token(lemma='horse', POS='NOUN'),
 Token(lemma='text', POS='NOUN'),
 Token(lemma='western', POS='ADJ'),
 Token(lemma='decision', POS='NOUN'),
 Token(lemma='true', POS='ADJ'),
 Token(lemma='friendly', POS='ADJ'),
 Token(lemma='file', POS='NOUN'),
 Token(lemma='choose', POS='VERB'),
 Token(lemma='discuss', POS='VERB'),
 Token(lemma='mention', POS='VERB')]

In [37]:
print("context words")
coocc_count.contexts_words[:20]

context words


["{'lemma': 'detonate', 'POS': 'VERB'}/nsubj",
 "{'lemma': 'guerrilla', 'POS': 'NOUN'}/nsubj",
 "{'lemma': 'hope', 'POS': 'VERB'}/advcl_nsubj",
 "{'lemma': 'tradeoff', 'POS': 'NOUN'}/nsubj",
 "{'lemma': 'move', 'POS': 'NOUN'}/nsubj",
 "{'lemma': 'suckup', 'POS': 'NOUN'}/acl_nsubj",
 "{'lemma': 'people', 'POS': 'NOUN'}/conj_nsubj",
 "{'lemma': 'suckup', 'POS': 'NOUN'}/nsubj",
 "{'lemma': 'people', 'POS': 'NOUN'}/nsubj",
 "{'lemma': 'remain', 'POS': 'VERB'}/nsubj_amod",
 "{'lemma': 'remain', 'POS': 'VERB'}/nsubj",
 "{'lemma': 'condemn', 'POS': 'VERB'}/nsubj",
 "{'lemma': 'condemn', 'POS': 'VERB'}/nsubj_case_of",
 "{'lemma': 'board', 'POS': 'NOUN'}/nsubj",
 "{'lemma': 'cleric', 'POS': 'NOUN'}/nsubj_case_of",
 "{'lemma': 'say', 'POS': 'VERB'}/nsubj_compound",
 "{'lemma': 'say', 'POS': 'VERB'}/ccomp_nsubj",
 "{'lemma': 'criminal', 'POS': 'ADJ'}/nsubj",
 "{'lemma': 'attack', 'POS': 'NOUN'}/nsubj",
 "{'lemma': 'give', 'POS': 'VERB'}/nsubj"]

In [35]:
coocc_count.shape

(11077, 15597)

In [45]:
i = 11
print("target word: ", target_vocabulary[i])
print("close words")
pprint.pprint(coocc_count.get_closest_words(target_vocabulary[i], 12))

target word:  Token(lemma='text', POS='NOUN')
close words
[(Token(lemma='drop', POS='NOUN'), 0.6407893959464501),
 (Token(lemma='diamond', POS='NOUN'), 0.6407893959464501),
 (Token(lemma='landmass', POS='NOUN'), 0.6407893959464501),
 (Token(lemma='nose', POS='NOUN'), 0.6407893959464501),
 (Token(lemma='tonne', POS='NOUN'), 0.6787122684390005),
 (Token(lemma='skin', POS='NOUN'), 0.7284623066791478),
 (Token(lemma='sample', POS='NOUN'), 0.7407620763173937),
 (Token(lemma='syllabary', POS='NOUN'), 0.792609661053915),
 (Token(lemma='cel', POS='NOUN'), 0.8203946979732251),
 (Token(lemma='commitment', POS='NOUN'), 0.8203946979732251),
 (Token(lemma='cabin', POS='NOUN'), 0.8203946979732251),
 (Token(lemma='listing', POS='NOUN'), 0.8203946979732251)]


### Weighting 

In [ ]:
ppmi = mangoes.weighting.PPMI()
svd = mangoes.reduction.SVD(dimensions=300)

In [ ]:
embeddings = mangoes.create_representation(coocc_count, weighting=ppmi, reduction=svd)

In [ ]:
embeddings.pprint(display=display)

In [ ]:
# Save the embeddings 
# embedding_path = os.path.join(OUTPUT_PATH,
#                               "embeddings/ppmi_svd_{}target_words_deprel".format(len(target_vocabulary)))

In [ ]:
# embeddings.save(embedding_path)

In [ ]:
# load embedding 
# embeddings = mangoes.Embeddings.load(embedding_path)

# Explore Embedding 

In [ ]:
import pandas as pd 

### Closest Words 

#### Similarities 

In [ ]:
sims = {0: "cityblock",
        1: "cosine", 
        2: "euclidean", 
        3: "l1", 
        4:"l2", 
        5:"manhattan",
        6:"braycurtis", 
        7:"canberra", 
        8:"chebyshev", 
        9:"correlation", 
        10:"dice", 
        11:"hamming", 
        12:"jaccard", 
        13:"kulsinski", 
        14:"mahalanobis", 
       15: "minkowski", 
       16: "rogerstanimoto", 
       17: "russellrao", 
       18: "seuclidean", 
        19:"sokalmichener", 
        20:"sokalsneath", 
       21: "sqeuclidean", 
       22: "yule"}


In [ ]:
i = 2
nb = 5

In [ ]:
result = {word: pd.Series([w for w, _ in embeddings.get_closest_words(word, nb=nb, metric=sims[i])], index=range(1,nb+1))
          for word in embeddings.words[100:200:10]}
print(f"similarity measure: {sims[i]}")
print(pd.DataFrame(result).transpose())

### Analogies 

In [ ]:
# Check what words are available 
question = "king queen male"
ans = "female"
for w in (question.split() + [ans]):
    try:
        embeddings.words.word_index[w]
        print(f"'{w}' exists")
    except KeyError:
        print(f"'{w}'  doesn't exist in embedding")

In [ ]:
# You can resolve analogy according to a representation using the analogy() method
# Here, we will display the results of some examples :
for analogy in [question]:
    print(analogy, '->', embeddings.analogy(analogy,5).using_cosadd)
    print(analogy, '->', embeddings.analogy(analogy,5).using_cosmul)

### Visualization

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import mangoes.visualize
plt.figure()

# 1. distances between the words
ax = plt.subplot(221, projection='polar')
mangoes.visualize.plot_distances(embeddings, ax)

# 2. isotropy
ax = plt.subplot(222)
mangoes.visualize.plot_isotropy(embeddings, ax)

# 3. t-sne
plt.subplot(212)
mangoes.visualize.plot_tsne(embeddings)

plt.show()


# Evalutaion 

In [ ]:
# # Evaluate
import mangoes.evaluation.analogy

google_dataset = mangoes.evaluation.analogy.GOOGLE
msr_dataset = mangoes.evaluation.analogy.MSR

analogy_evaluation = mangoes.evaluation.analogy.Evaluation(embeddings, google_dataset, msr_dataset)
print(analogy_evaluation.get_report())